# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 2, Evaluation

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
wmg.Init("LinslvPerf_ConstPoissonMpi1");
wmg.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'LinslvPerf_ConstPoissonMpi1'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1'.


In [3]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [4]:
databases

#0: { Session Count = 37; Grid Count = 7; Path = \\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1 }


In [5]:
wmg.Sessions

#0: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k5_exp_Kcycle_schwarz	11/05/2021 09:34:44	a0cea6c5...
#1: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k3_exp_Kcycle_schwarz	11/05/2021 09:26:27	32d21dd8...
#2: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k5_exp_gmres_levelpmg	11/05/2021 10:02:50	d5c299c5...
#3: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k5_exp_Kcycle_schwarz	11/05/2021 09:33:45	46fe6be0...
#4: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k3_exp_gmres_levelpmg	11/05/2021 09:49:56	f1faadff...
#5: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k5_exp_gmres_levelpmg	11/05/2021 10:02:20	778aa611...
#6: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_Kcycle_schwarz	11/05/2021 09:24:36	cee50db8...
#7: LinslvPerf_ConstPoissonMpi1	SIP_J2560_k5_exp_gmres_levelpmg	11/05/2021 10:01:08	459bc146...
#8: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_gmres_levelpmg	11/05/2021 09:40:30	27192c60...
#9: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_Kcycle_schwarz	11/05/2021 09:22:59	8ead93f7...
#10: LinslvPerf_ConstPoisso

In [6]:
using System.IO;
using BoSSS.Application.SipPoisson;

## Create Table for Post-Processing

In [7]:
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}

In [8]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);

In [9]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [10]:
//wmg.Sessions[0].KeysAndQueries["NoIter"].GetType()

In [11]:
//wmg.Sessions[0].Name

Select those columns which are of interest:

In [12]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter");

In [13]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

In [14]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [15]:
SubTab.GetColumnNames()

index,value
0,SessionName
1,DGdegree:T
2,Grid:NoOfCells
3,LinearSolver.SolverCode
4,DOFs
5,MatrixAssembly
6,Aggregation_basis_init_time
7,Solver_Init_time
8,Solver_Run_time
9,Solver_InitAndRun_time


Available DG degrees:

In [16]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree:T").ToSet().OrderBy(s => s).ToArray();
DGdegrees

index,value
0,2
1,3
2,5


All used solvers:

In [17]:
SubTab.GetColumn<int>("LinearSolver.SolverCode").ToSet().Select(i => (LinearSolverCode)i).ToArray()

index,value
0,exp_Kcycle_schwarz
1,exp_gmres_levelpmg
2,classic_pardiso


In [18]:
//RuntimePlot.PlotNow();

In [19]:
//using SolverCodes = BoSSS.Solution.Control.LinearSolverConfig.Code;

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [20]:
string SlvCode2Name(LinearSolverCode solver_name) {
        switch(solver_name) {
           case LinearSolverCode.classic_pardiso: 
               return "Pardiso"; 
           case LinearSolverCode.exp_gmres_levelpmg: 
               return "GMRES w p2G"; 
           case LinearSolverCode.exp_Kcycle_schwarz: 
               return "OrthoMG w Add Swz"; 
           default:
               Console.WriteLine("unknown: " + solver_name); 
               return "unknown";
        } 
}

In [21]:
PlotFormat SlvCode2Pltfmt(LinearSolverCode solver_name) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case LinearSolverCode.classic_pardiso: 
            Fmt.PointType = PointTypes.OpenCircle;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case LinearSolverCode.exp_gmres_levelpmg: 
            Fmt.PointType = PointTypes.Box;
            break;
        case LinearSolverCode.exp_Kcycle_schwarz: 
            Fmt.PointType = PointTypes.LowerTriangle;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.5;
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = LineColors.Black;
    Fmt.LineWidth = 3;
    return Fmt;
}

In [22]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax) {

int rows    = DGdegrees.Length;
int columns = 1;
//LinearSolverCode[] ignore_solvers = {LinearSolverCode.classic_pardiso, LinearSolverCode.classic_cg, LinearSolverCode.exp_softpcg_schwarz, LinearSolverConfig.Code.exp_direct_lapack};
LinearSolverCode[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree:T"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           LinearSolverCode solver_name = (LinearSolverCode)Convert.ToInt32(Row["LinearSolver.SolverCode"]);
           //ignore the solvers specified in ingore_solvers
           foreach(LinearSolverCode sc in ignore_solvers){
               if(solver_name==sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           // ===!!!=== CAUTION ===!!!===
           // Solverframework (SolverEnum numbers) have changed! Please check LinearSolver and NonLinearSolver for details
           
           //process the other solvers
           Fmt = SlvCode2Pltfmt(solver_name);
           Nmn = SlvCode2Name(solver_name);
       });
    
       double[] dof = new[]{1e3,1e6};
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == 0) {
          multiplots[iRow,iCol].ShowLegend = true;
          multiplots[iRow,iCol].LegendAlignment = new string[]{"i","t","l"};
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [23]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top left Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 Pardiso 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 linear

In [24]:
multiplots[0,0].Regression()

index,Key,Value
0,GMRES w p2G,1.0488198615162936
1,OrthoMG w Add Swz,1.3635498520131717
2,Pardiso,1.5004401369462375
3,linear,1


In [25]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 2");

In [26]:
multiplots[1,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz,1.3818574097037837
1,GMRES w p2G,1.0545712027380143
2,Pardiso,1.6130434219928662
3,linear,1


In [27]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 3");

In [28]:
multiplots[2,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz,1.4986695047515197
1,GMRES w p2G,0.9572093639383554
2,Pardiso,1.5833392233854404
3,linear,1


In [29]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 5");

Experimental: Latex Preview:

In [30]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
multiplots.PlotCairolatex().PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top left Left reverse 


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 Z M 2.484375 0 "/>
 
 
<path style="stroke:none;" d="M 5.90625 -0.625 C 6.046875 -0.40625 6.4375 -0.015625 6.546875 -0.015625 C 6.640625 -0.015625 6.640625 -0.09375 6.640625 -0.234375 L 6.640625 -1.96875 C 6.640625 -2.359375 6.671875 -2.40625 7.328125 -2.40625 L 7.328125 -2.71875 C 6.953125 -2.71875 6.40625 -2.6875 6.109375 -2.6875 C 5.71875 -2.6875 4.859375 -2.6875 4.5 -2.71875 L 4.5 -2.40625 L 4.828125 -2.40625 C 5.71875 -2.40625 5.75 -2.296875 5.75 -1.9375 L 5.75 -1.296875 C 5.75 -0.171875 4.484375 -0.09375 4.203125 -0.09375 C 3.5625 -0.09375 1.578125 -0.4375 1.578125 -3.40625 C 1.578125 -6.390625 3.546875 -6.71875 4.140625 -6.71875 C 5.21875 -6.71875 6.125 -5.828125 6.3125 -4.359375 C 6.34375 -4.21875 6.34375 -4.1875 6.484375 -4.1875 C 6.640625 -4.1875 6.640625 -4.21875 6.640625 -4.421875 L 6.640625 -6.78125 C 6.640625 -6.953125 6.640625 -7.03125 6.53125 -7.03125 C 6.484375 -7.03125 6.453125 -7.03125 6.375 -6.90625 L 5.875 -6.171875 C 5.546875 -6.484375 5.015625 -7.03125 4.03125 -7.03125 C 2.171875 -7.03125 0.5625 -5.453125 0.5625 -3.40625 C 0.5625 -1.359375 2.15625 0.21875 4.046875 0.21875 C 4.78125 0.21875 5.578125 -0.046875 5.90625 -0.625 Z M 5.90625 -0.625 "/>
 
 
<path style="stroke:none;" d="M 2.40625 -6.59375 C 2.3125 -6.8125 2.28125 -6.8125 2.046875 -6.8125 L 0.375 -6.8125 L 0.375 -6.5 L 0.609375 -6.5 C 1.375 -6.5 1.390625 -6.390625 1.390625 -6.03125 L 1.390625 -1.046875 C 1.390625 -0.78125 1.390625 -0.3125 0.375 -0.3125 L 0.375 0 C 0.71875 -0.015625 1.203125 -0.03125 1.53125 -0.03125 C 1.859375 -0.03125 2.359375 -0.015625 2.703125 0 L 2.703125 -0.3125 C 1.671875 -0.3125 1.671875 -0.78125 1.671875 -1.046875 L 1.671875 -6.421875 L 1.6875 -6.421875 L 4.09375 -0.21875 C 4.140625 -0.09375 4.1875 0 4.28125 0 C 4.390625 0 4.421875 -0.078125 4.46875 -0.1875 L 6.921875 -6.5 L 6.921875 -0.78125 C 6.921875 -0.421875 6.90625 -0.3125 6.140625 -0.3125 L 5.90625 -0.3125 L 5.90625 0 C 6.265625 -0.03125 6.953125 -0.03125 7.34375 -0.03125 C 7.71875 -0.03125 8.390625 -0.03125 8.765625 0 L 8.765625 -0.3125 L 8.515625 -0.3125 C 7.75 -0.3125 7.734375 -0.421875 7.734375 -0.78125 L 7.734375 -6.03125 C 7.734375 -6.390625 7.75 -6.5 8.515625 -6.5 L 8.765625 -6.5 L 8.765625 -6.8125 L 7.078125 -6.8125 C 6.8125 -6.8125 6.8125 -6.796875 6.75 -6.625 L 4.5625 -1 Z M 2.40625 -6.59375 "/>
 
 
<path style="stroke:none;" d="M 2.234375 -3.515625 L 2.234375 -6.09375 C 2.234375 -6.328125 2.234375 -6.453125 2.453125 -6.484375 C 2.546875 -6.5 2.84375 -6.5 3.046875 -6.5 C 3.9375 -6.5 5.046875 -6.453125 5.046875 -5.015

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [31]:
var multiplots = PlotSolverBehave("NoIter", false, 0, 140);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top left Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 Pardiso 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 linear

In [32]:
multiplots[0,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G,False,False,"[ 3200, 25600, 204800, 691200, 1638400 ]","[ 64, 68, 71, 74, 76 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz,False,False,"[ 3200, 25600, 204800, 691200, 1638400 ]","[ 1, 1, 8, 12, 12 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,Pardiso,False,False,"[ 3200, 25600, 204800, 691200 ]","[ 1, 1, 1, 1 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,linear,False,False,"[ 1000, 1000000 ]","[ 1, 1000 ]"


In [33]:
multiplots[1,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz,False,False,"[ 6400, 51200, 409600, 1382400, 3276800 ]","[ 1, 5, 8, 11, 11 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G,False,False,"[ 6400, 51200, 409600, 1382400, 3276800 ]","[ 84, 88, 96, 100, 103 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,Pardiso,False,False,"[ 6400, 51200, 409600 ]","[ 1, 1, 1 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,linear,False,False,"[ 1000, 1000000 ]","[ 1, 1000 ]"


In [34]:
multiplots[2,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz,False,False,"[ 17920, 143360, 1146880, 3870720 ]","[ 1, 9, 10, 11 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G,False,False,"[ 17920, 143360, 1146880, 3870720 ]","[ 129, 130, 138, 142 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,Pardiso,False,False,"[ 17920, 143360 ]","[ 1, 1 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,linear,False,False,"[ 1000, 1000000 ]","[ 1, 1000 ]"


### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [35]:
var multiplots = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top left Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 Pardiso 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz 
 
 
 
 
 
 
 
 
 GMRES w p2G 
 
 
 
 
 
 
 
 Pardiso 
 
 
 
 
 
 linear

In [36]:
multiplots[0,0].Regression()

index,Key,Value
0,GMRES w p2G,1.0349339612416384
1,OrthoMG w Add Swz,0.9710818987017887
2,Pardiso,1.5811245685090018
3,linear,1


In [37]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 2");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[0,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_Kcycle_schwarz)).Value,
   1.1,
   "Ortho MG scaling out of linear for p = 2");

In [38]:
multiplots[1,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz,1.096272655047665
1,GMRES w p2G,1.048720643952981
2,Pardiso,1.7402025916342978
3,linear,1


In [39]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 3");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[1,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_Kcycle_schwarz)).Value,
   1.1,
   "Ortho MG scaling out of linear for p = 3");

In [40]:
multiplots[2,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz,1.1791270390051172
1,GMRES w p2G,0.9475671728865691
2,Pardiso,1.8096862074686522
3,linear,1


In [41]:
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_gmres_levelpmg)).Value,
   1.1,
   "2-plevel GMRES scaling out of linear for p = 5");
NUnit.Framework.Assert.LessOrEqual(
   multiplots[2,0].Regression().Single(tt =>  tt.Key == SlvCode2Name(LinearSolverCode.exp_Kcycle_schwarz)).Value,
   1.2,
   "Ortho MG scaling out of linear for p = 5");

## Appendix: Table Form of Extracted Data

In [42]:
SubTab.Print()

    SessionName                       DGdegree:T Grid:NoOfCells LinearSolver.SolverCode DOFs    MatrixAssembly Aggregation_basis_init_time Solver_Init_time Solver_Run_time Solver_InitAndRun_time NoIter Solver_TimePerIter   
0:  SIP_J69120_k5_exp_Kcycle_schwarz  5          69120          41                      3870720 1595.5750317   370.5485913                 365.2337363      22326.0650855   23061.847413099997     11     2029.6422805         
1:  SIP_J163840_k3_exp_Kcycle_schwarz 3          163840         41                      3276800 389.6183957    69.2043143                  121.0330343      5226.8886179    5417.1259665           11     475.17169253636365   
2:  SIP_J69120_k5_exp_gmres_levelpmg  5          69120          47                      3870720 1184.4338351   364.5580224                 95.8446894       850.2855715     1310.6882833           142    5.9879265598591545   
3:  SIP_J20480_k5_exp_Kcycle_schwarz  5          20480          41                      1146880 511.5904

35: SIP_J2560_k2_classic_pardiso      2          2560           2                       25600   2.3956543      0.242008                    0.3827033        1.8572232       2.4819345              1      1.8572232            
36: SIP_J320_k2_classic_pardiso       2          320            2                       3200    0.3811242      0.0475136                   0.0656449        0.2136681       0.3268266              1      0.2136681            